In [1]:
import plotly.graph_objects as go
import pandas as pd
from numpy import nan
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc


athelet_info_lzx = pd.read_csv('athlete_events.csv')
noc_regions_lzx = pd.read_csv('noc_regions.csv')
athelet_info_lzx['NOC']= athelet_info_lzx['NOC'].replace('SGP','SIN')
noc_regions_lzx['region']= noc_regions_lzx['region'].replace('Boliva','Bolivia')

In [2]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [3]:
app.layout = html.Div(children=[
    html.H1(children = 'Medals'),

    html.H2(children = 'Select a Medal type:'),
    dcc.Dropdown(
                id='medals',
                options=[{'label': i, 'value': i} for i in ['All','Gold','Silver','Bronze']],
                value='All',
                clearable = False,
                searchable = False,
                style = {'width':'50%'}
            ),
    dcc.Graph(
        id='choropleth',
        style = {'width':'50%'}
    )
])

In [4]:
def update_dataframe(medal_type):
    if(medal_type == 'All'):
        NOC_group = pd.DataFrame(athelet_info_lzx.groupby(['NOC'])['Medal'].count())
        return pd.merge(NOC_group,noc_regions_lzx,how='inner',on=[NOC_group.index, noc_regions_lzx.NOC])
    else:
        temp = pd.DataFrame(athelet_info_lzx[athelet_info_lzx.Medal == medal_type][['NOC','Medal']].groupby(['NOC']).count()).reset_index()
        result = pd.merge(noc_regions_lzx,temp, how='left', on='NOC')
        result.Medal = result.Medal.fillna(0)
        return result

@app.callback(
    dash.dependencies.Output('choropleth','figure'),
    dash.dependencies.Input('medals','value'))
def update_graph(medal_type):
    
    data = update_dataframe(medal_type)    
    color = ''
    tick_step = 0
    reverse = True
    
    if (medal_type == 'All'):
        color = 'Purples'
        tick_step = 500

    elif (medal_type == 'Gold'):
        color = 'solar'
        tick_step = 200
        reverse = False

    elif (medal_type == 'Silver'):
        color = 'Greys'
        tick_step = 200

    elif (medal_type == 'Bronze'):
        color = 'OrRd'
        tick_step = 200

    return go.Figure(data=go.Choropleth(
    locations = data.region,
    z = data['Medal'],
    text = data.NOC,
    colorscale = color,
    autocolorscale=False,
    reversescale=reverse,
    marker_line_color='darkgray',
    colorbar_title = 'The Number of Medals',
    locationmode = 'country names',
    
    )).update_traces(
    colorbar = dict(
        ticks = 'outside',
        tickmode = 'linear',
        tick0 = 0,
        dtick = tick_step
    ),
        
    ).update_layout(
    title_text='The Number of '+ medal_type + ' Medals Won By Each Country in Olympic Games From 1896 - 2016',
    height = 540,
    width = 960,
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular',
    ),
    annotations = [dict(
        x=0.5,
        y=-0.1,
        xref='paper',
        yref='paper',
        text='Data Source: <a href="https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results">\
            Kaggle</a>',
        showarrow = False
    )])
    


In [5]:
if __name__ == '__main__':
    app.run_server( mode = 'external',debug = True, port= 8052)

Dash app running on http://127.0.0.1:8052/
